<div style="text-align: left; background-color:#FFFFCC; font-family:Arial; color:#526085; padding: 12px; line-height:1.25;border-radius:1px; margin-bottom: 0em; text-align: center; font-size: 58px;border-style: solid;border-color: dark green;">Lab 02: Working with time-series data</div>
<div style="text-align: left; background-color:#99FFFF; font-family: Arial; color:#551184; padding: 12px; line-height:1.25;border-radius:1px; margin-bottom: 0em; text-align: center; font-size: 18px">Bộ môn: Trực quan hóa dữ liệu | Nhóm 8</div>

# <div style="text-align: left; background-color:#0C6A86; font-family:Arial; color:white; padding: 12px; line-height:1.25;border-radius:1px; margin-bottom: 0em; text-align: center; font-size: 30px;border-style: solid;border-color: black;">PHẦN 1. Tiền xử lý & khám phá dữ liệu</div>

# Import các thư viện cần thiết

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

import plotly.express as px
import plotly.graph_objects as go

# Nội dung chính

## Đọc dữ liệu

In [14]:
df = pd.read_csv("Data/data_18_19_processed.csv")
df.head()

,Reference Number,Grid Ref: Easting,Grid Ref: Northing,Number of Vehicles,1st Road Class,1st Road Class & No,Road Surface,Lighting Conditions,Weather Conditions,Local Authority,Type of Vehicle,Casualty Class,Casualty Severity,Sex of Casualty,Age of Casualty,Full Time
0,51B0230,433936,428874,1,6,U,1,4,1,E08000035,9,3,3,2,87,2018-01-11 07:00:00
1,51B0349,430076,433872,1,6,U,2,1,1,E08000035,11,2,3,2,60,2018-01-11 08:55:00
2,51B0349,430076,433872,1,6,U,2,1,1,E08000035,11,2,3,2,51,2018-01-11 08:55:00
3,51B0349,430076,433872,1,6,U,2,1,1,E08000035,11,2,3,2,50,2018-01-11 08:55:00
4,51B0349,430076,433872,1,6,U,2,1,1,E08000035,11,2,3,2,51,2018-01-11 08:55:00


### Biểu đồ

##### **Tiêu đề**: thống kê số các tai nạn theo nhóm tuổi và giới tính

##### **Loại biểu đồ**: Biểu đồ hình tháp

#### **Lý do chọn**: 
- **Trực quan hóa dễ hiểu**: Biểu đồ tháp là một công cụ rất thích hợp để biểu diễn và so sánh theo độ tuổi giữa 2 nhóm cá thể một cách trực quan nhất, giúp người nhìn có thể nhanh chóng nắm bắt thông tin.
- **Dễ dàng xác định được kết quả cần so sánh**: dựa vào đồ thị có thể dễ dàng thấy được sự chênh lệch khi thực hiện so sánh một giá trị nào đó liên quan đến nội dung đồ thị

##### Trực quan hóa

In [18]:
df['Sex of Casualty'] = df['Sex of Casualty'].replace({1: 'Male', 2: 'Female'})
bins = [1, 16, 30, 50, 65, 80, 95]
labels = ['1-16','16-30', '30-50', '50-65','65-80','80-95']
df['Age Group'] = pd.cut(df['Age of Casualty'], bins=bins, labels=labels, right=False)
agesex_df = df.pivot_table(index='Sex of Casualty', columns='Age Group', aggfunc='size', fill_value=0)
age_groups = agesex_df.columns
male_counts = agesex_df.loc['Male']
female_counts = agesex_df.loc['Female'] * -1  

fig = go.Figure()
fig.add_trace(go.Bar(
    y=age_groups,
    x=male_counts,
    name='Male',
    orientation='h',
    marker=dict(color='#800000')
))
fig.add_trace(go.Bar(
    y=age_groups,
    x=female_counts,
    name='Female',
    orientation='h',
    marker=dict(color='purple')
))
fig.update_layout(
    title='Population Pyramid by Age Group and Sex',
    barmode='overlay',
    bargap=0.1,
    xaxis=dict(
        title='Number of people',
        tickvals=[-max(abs(female_counts)), 0, max(male_counts)],
        ticktext=[max(abs(female_counts)), 0, max(male_counts)],
    ),
    yaxis=dict(title='Age Group'),
    template='plotly_white'
)
fig.show()

C:\Users\Lyliths\AppData\Local\Temp\ipykernel_12112\474487751.py:5: FutureWarning:

The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior



#### **Kỹ thuật trực quan được sử dụng**:
- **Màu sắc**: Sử dụng 2 màu khác nhau để biểu diễn cho đồ thị, giúp cho người xem có thể phân biệt nhanh chóng giữa 2 cá thể được đề cập trong đồ thị, ngoài ra còn giúp đồ thị rõ ràng hơn
- **Kỹ thuật Manipulate View**:
    - **Hiển thị thông tin qua tooltip**: Khi đưa con trỏ chuột vào một thành phần của đồ thị, nó sẽ hiện lên số lượng cá thể và nhóm tuổi của cá thể đó

⇒ Đồ thị trực quan, rõ ràng, dễ nắm bắt thông tin cần thiết một cách nhanh chóng và dễ tương tác với người dùng

### Nhận xét: 
- Dựa vào đồ thị, ta dễ dàng thấy được số lần gặp tai nạn trải dài ở các độ tuổi từ 1 đến 95 tuổi. 
- Đa số nam giới sẽ gặp tai nạn nhiều hơn so với nữ giới. Ở độ tuổi 30-50, số lượng nam giới gặp tai nạn gần gấp đôi so với nữ giới, tương tự với nhóm tuổi 16-30 tuổi.
- Độ tuổi có số lần tai nạn ít nhất là 80-95 tuổi ở cả nam và nữ, độ tuổi có số lượng tai nạn nhiều nhất là 30-50 tuổi, kế đến là từ 16-30 tuổi.